# 🚀 Parts of Speech Game - LLM API Server

This notebook sets up an API server for generating sentences using LLM models.

## 📋 Instructions:
1. Run all cells in order
2. Copy the ngrok URL from the output
3. Use this URL in your local Streamlit app

## ⚠️ Important Notes:
- Keep this notebook running while using the local app
- The ngrok URL changes each time you restart
- Free ngrok has usage limits

In [ ]:
# Install required packages
!pip install flask flask-cors pyngrok transformers torch openai

# Install PyThaiNLP for Thai language support
!pip install pythainlp

In [ ]:
# Set up ngrok authentication (optional but recommended)
# Get your auth token from https://ngrok.com/

from pyngrok import ngrok

# Uncomment and add your ngrok auth token for better stability
# ngrok.set_auth_token("2xdXSqaLA8wPwljc2izXFVfaqxq_3vL4WrgXXJ8zo6UM6CugP")

print("✅ Ngrok setup complete")

In [ ]:
# Create the API server code

api_server_code = '''
# Google Colab API Server for Parts of Speech Game
# This file should be run in Google Colab to serve the LLM API

import os
from flask import Flask, request, jsonify
from flask_cors import CORS
import threading
import time
from pyngrok import ngrok
import openai
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests

# Global variables for models
llm_pipeline = None
tokenizer = None
model = None

# Configuration
CONFIG = {
    'model_name': 'microsoft/DialoGPT-medium',  # You can change this to other models
    'max_length': 100,
    'temperature': 0.7,
    'use_openai': False,  # Set to True if you want to use OpenAI API
    'openai_api_key': None  # Set your OpenAI API key here
}

def initialize_model():
    """Initialize the language model"""
    global llm_pipeline, tokenizer, model
    
    print("Initializing language model...")
    
    if CONFIG['use_openai'] and CONFIG['openai_api_key']:
        # Use OpenAI API
        openai.api_key = CONFIG['openai_api_key']
        print("Using OpenAI API")
    else:
        # Use Hugging Face transformers
        try:
            device = 0 if torch.cuda.is_available() else -1
            print(f"Using device: {'GPU' if device == 0 else 'CPU'}")
            
            # Load model and tokenizer
            tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_name'])
            model = AutoModelForCausalLM.from_pretrained(CONFIG['model_name'])
            
            # Create text generation pipeline
            llm_pipeline = pipeline(
                'text-generation',
                model=model,
                tokenizer=tokenizer,
                device=device,
                max_length=CONFIG['max_length'],
                temperature=CONFIG['temperature']
            )
            print(f"Model {CONFIG['model_name']} loaded successfully")
            
        except Exception as e:
            print(f"Error loading model: {e}")
            # Fallback to a smaller model
            try:
                llm_pipeline = pipeline('text-generation', model='gpt2')
                print("Fallback to GPT-2 model")
            except Exception as e2:
                print(f"Error loading fallback model: {e2}")
                llm_pipeline = None

# Sample sentences as fallback
FALLBACK_SENTENCES = {
    'en': {
        'easy': [
            "The cat sleeps peacefully.",
            "She runs quickly today.",
            "Birds fly high above.",
            "We eat delicious food."
        ],
        'medium': [
            "The beautiful flowers bloom in spring garden.",
            "Students study hard for their important exams.",
            "My grandmother tells interesting stories every evening."
        ],
        'hard': [
            "The magnificent orchestra performed brilliantly at the prestigious concert hall last night.",
            "Scientists carefully analyze complex data to understand mysterious phenomena in deep space."
        ]
    },
    'th': {
        'easy': [
            "แมวนอนหลับสบาย",
            "เขาวิ่งเร็วมาก",
            "นกบินสูงขึ้น",
            "เรากินข้าวอร่อย"
        ],
        'medium': [
            "นักเรียนขยันอ่านหนังสือเพื่อสอบ",
            "ดอกไม้สวยบานในสวนหลังบ้าน",
            "คุณยายเล่านิทานให้ฟังทุกคืน"
        ],
        'hard': [
            "นักวิทยาศาสตร์วิเคราะห์ข้อมูลซับซ้อนเพื่อทำความเข้าใจปรากฏการณ์ลึกลับในอวกาศ",
            "วงดุริยางค์ชื่อดังแสดงอย่างยอดเยี่ยมในหอประชุมใหญ่เมื่อคืนที่ผ่านมา"
        ]
    }
}

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'model_loaded': llm_pipeline is not None or CONFIG['use_openai'],
        'timestamp': time.time()
    })

@app.route('/generate_sentence', methods=['POST'])
def generate_sentence_api():
    """API endpoint to generate sentences"""
    try:
        data = request.get_json()
        
        # Extract parameters
        language = data.get('language', 'en')
        difficulty = data.get('difficulty', 'easy')
        
        # Fallback to sample sentences
        import random
        sentences = FALLBACK_SENTENCES.get(language, {}).get(difficulty, [])
        if sentences:
            sentence = random.choice(sentences)
        else:
            sentence = "The quick brown fox jumps." if language == 'en' else "แมวดำวิ่งเร็ว"
        
        return jsonify({
            'success': True,
            'sentence': sentence,
            'language': language,
            'difficulty': difficulty,
            'method': 'fallback'
        })
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

def run_server():
    """Run the Flask server"""
    app.run(host='0.0.0.0', port=5000, debug=False)

def setup_ngrok():
    """Setup ngrok tunnel for external access"""
    try:
        # Create tunnel
        public_url = ngrok.connect(5000)
        print(f"\\n🌐 Public URL: {public_url}")
        print(f"📱 Use this URL in your Streamlit app: {public_url}")
        print(f"🔗 API endpoint: {public_url}/generate_sentence")
        
        return public_url
        
    except Exception as e:
        print(f"Error setting up ngrok: {e}")
        return None

if __name__ == '__main__':
    print("🚀 Starting Parts of Speech Game API Server")
    
    print("🌐 Setting up ngrok tunnel...")
    public_url = setup_ngrok()
    
    print("\n🎯 API Endpoints:")
    print("  - POST /generate_sentence - Generate sentences")
    print("  - GET /health - Health check")
    
    print("\n🔥 Starting Flask server...")
    
    # Start the server
    try:
        run_server()
    except KeyboardInterrupt:
        print("\n👋 Server stopped")
        ngrok.disconnect(public_url)
        ngrok.kill()
'''

# Write the server code to a file
with open('api_server.py', 'w', encoding='utf-8') as f:
    f.write(api_server_code)

print("✅ API server code created successfully!")

In [ ]:
# Run the API server

print("🚀 Starting the API server...")
print("⚠️  Keep this cell running while using the local Streamlit app")
print("📋 Copy the ngrok URL from the output below")
print("" + "=" * 50)

# Execute the server
exec(open('fixed_colab_api_server_new.py').read())

## 📝 Usage Instructions

1. **Copy the ngrok URL** from the output above (looks like `https://xxxx-xx-xx-xx-xx.ngrok-free.app`)

2. **In your local Streamlit app:**
   - Check "Use LLM API"
   - Paste the ngrok URL
   - Click "Test API Connection"
   - Start playing the game!

3. **Keep this notebook running** while using the local app

## 🔧 Customization

You can modify the `CONFIG` dictionary in the server code to:
- Use different models
- Add OpenAI API support
- Adjust generation parameters

## ⚠️ Important Notes

- **Free ngrok limits:** 1 tunnel, 40 connections/minute
- **Colab timeout:** Sessions may timeout after inactivity
- **URL changes:** New ngrok URL each time you restart
- **HTTPS required:** Some browsers require HTTPS for API calls